# 残差网络

残差块（Residual Block）是残差网络（ResNet）的基本结构，它通过引入快捷连接（shortcut connections）来缓解深度神经网络中的梯度消失问题。残差块的主要思想是让网络学习输入和输出之间的残差函数，而不是直接学习输入到输出的映射。

In [1]:
import d2lzh as d2l
from mxnet.gluon import nn
from mxnet import init, nd,gluon

class Residual(nn.Block):
    def __init__(self,num_channels,use_1x1conv=False,strides=1,**kwargs):
        super(Residual,self).__init__(**kwargs)
        self.conv1=nn.Conv2D(num_channels,kernel_size=3,padding=1,strides=strides)
        self.conv2=nn.Conv2D(num_channels,kernel_size=3,padding=1)
        if use_1x1conv:
            self.conv3=nn.Conv2D(num_channels,kernel_size=1,strides=strides)
        else:
            self.conv3=None
        self.bn1=nn.BatchNorm()
        self.bn2=nn.BatchNorm()

    def forward(self,X):
        Y=nd.relu(self.bn1(self.conv1(X)))
        Y=self.bn2(self.conv2(Y))
        if self.conv3:
            X=self.conv3(X)
        return nd.relu(Y+X)

In [3]:
blk=Residual(3)
blk.initialize()
X=nd.random.normal(shape=(4,3,6,6)) # 4个样本，3个通道，6*6的图像
blk(X).shape

(4, 3, 6, 6)

In [4]:
blk=Residual(6,use_1x1conv=True,strides=2) # 通道数从3变为6，步幅为2
blk.initialize()
blk(X).shape

(4, 6, 3, 3)

ResNet模型

In [5]:
net=nn.Sequential()
net.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3),
       nn.BatchNorm(),
       nn.Activation('relu'),
       nn.MaxPool2D(pool_size=3,strides=2,padding=1))

In [6]:
def resnet_block(num_channels,num_residuals,first_block=False):
    blk=nn.Sequential()
    for i in range(num_residuals):
        if i==0 and not first_block:
            blk.add(Residual(num_channels,use_1x1conv=True,strides=2)) 
        else:
            blk.add(Residual(num_channels))
    return blk

In [7]:
net.add(resnet_block(64,2,first_block=True),
       resnet_block(128,2),
       resnet_block(256,2),
       resnet_block(512,2))
net.add(nn.GlobalAvgPool2D(),
       nn.Dense(10)) # 加入全局平均池化层和全连接层

In [8]:
net.initialize()
X=nd.random.uniform(shape=(1,1,224,224))
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv7 output shape:	 (1, 64, 112, 112)
batchnorm6 output shape:	 (1, 64, 112, 112)
relu0 output shape:	 (1, 64, 112, 112)
pool0 output shape:	 (1, 64, 56, 56)
sequential1 output shape:	 (1, 64, 56, 56)
sequential2 output shape:	 (1, 128, 28, 28)
sequential3 output shape:	 (1, 256, 14, 14)
sequential4 output shape:	 (1, 512, 7, 7)
pool1 output shape:	 (1, 512, 1, 1)
dense0 output shape:	 (1, 10)


获取数据和训练模型
...